In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
llm(prompt="Explain me the difference between Data Lakehouse and Data Warehouse.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' Unterscheidung between data lakehouse and data warehouse is a common topic of discussion in the data engineering community, as both are designed to store large amounts of data but have different architectures, use cases, and benefits. A data lakehouse is a centralized repository that stores all the data from various sources in its raw form, without transforming or processing it. On the other hand, a data warehouse is a structured repository that stores data in a specific format, typically after cleaning, transforming, and organizing it.\n\nHere are some key differences between a data lakehouse and a data warehouse:\n\n1. Data Structure: A data lakehouse stores data in its raw form, including semi-structured and unstructured data, while a data warehouse stores data in a structured format, typically in a relational database management system (RDBMS).\n2. Data Processing: A data lakehouse does not process or transform data, whereas a data warehouse often performs ETL (Extract, Transform

In [ ]:
from langchain.document_loaders import WebBaseLoader

web_links = ["https://www.uts.edu.my/about-university/","https://en.wikipedia.org/wiki/University_of_Technology_Sarawak", "https://www.uts.edu.my/school/", "https://scm.uts.edu.my/programme/undergraduate-programme/bachelor-of-computer-science/", "https://www.uts.edu.my/management-team/", "https://www.uts.edu.my/facility/", "https://www.uts.edu.my/tuition-fees/"]

loader = WebBaseLoader(web_links)
documents = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# storing embeddings in the vector store
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [ ]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)

In [ ]:
chat_history = []

query = "ucts is uts?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])
print(chat_history)

 Yes, UCTS is now known as UTS after it was upgraded to a full-fledged university in 2021.
[]


In [ ]:
chat_history = [(query, result["answer"])]

query = "what is full name of ucts and uts?"
result = chain({"question": query, "chat_history": chat_history})

print(result['answer'])
print(chat_history)

  The full name of UCTS is University College of Technology Sarawak.
[('ucts is uts?', ' Yes, UCTS is now known as UTS after it was upgraded to a full-fledged university in 2021.')]


In [ ]:
print(result['source_documents'])

[Document(page_content='History[edit]\nThe university was established on 1 April 2013 as "University College of Technology Sarawak" (UCTS).[5] It is the first university in Sarawak owned by the state government.[6][7] UTS is considered as one of the newest and latest university in Malaysia. The university is recognised as the only TVET university in Sarawak with the aim of producing highly educated and skilled professionals for new and rapidly developing industries.[8][9] The university held its inaugural convocation on 8 October 2016 with 100 graduates. During the occasion, UTS announced its first chancellor, the Governor of Sarawak, Tun Pehin Sri Abdul Taib Mahmud.[10]', metadata={'source': 'https://en.wikipedia.org/wiki/University_of_Technology_Sarawak', 'title': 'University of Technology Sarawak - Wikipedia', 'language': 'en'}), Document(page_content='^ "UCTS runs transfer degree programme with 4 universities". Borneo Post Online. 8 November 2018. Retrieved 26 September 2023.\n\n^ 

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
 return "UTF-8"
locale.getpreferredencoding = getpreferredencoding